<a href="https://colab.research.google.com/github/RifatMuhtasim/Data_Science_Workflow/blob/main/3.4.Handle_Outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Identify Outliers

In [ ]:
# View outliers using bloxplot
fig, axes = plt.subplots(4, 1, figsize=(6,8)) # Rows: 4, Column: 1

# Plot boxplots using Seaborn
sns.boxplot(data= df, x="sepal length (cm)", ax=axes[0])
sns.boxplot(data= df, x="sepal width (cm)", ax=axes[1])
sns.boxplot(data= df, x="petal length (cm)", ax=axes[2])
sns.boxplot(data= df, x="petal width (cm)", ax=axes[3])

plt.tight_layout()
plt.show()

In [ ]:
# Boxplot For Multiple Columns

def Box_plot_multiple_columns(df):
    # numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    numerical_cols = df.select_dtypes(include=['number']).columns
    fig, axes = plt.subplots(len(numerical_cols), 1, figsize=(10, 12))

    # Plot box plots for each feature
    for index, col in enumerate(numerical_cols):
        sns.boxplot(data= df, x=col, ax=axes[index])
        axes[index].set_title(f'Box Plot of {col.capitalize()}')
        axes[index].set_xlabel(col.capitalize())
        axes[index].set_ylabel('Value')

    plt.tight_layout()
    plt.show()


Box_plot_multiple_columns(df= df)

# 2. Remove Outliers

In [ ]:
# Remove Outliers using IQR for multiple column

def Remove_outliers_using_IQR(df):
    outliers_index_list = []

    for col in df.columns:
        if (df[col].dtypes == "float") | (df[col].dtypes == "int")  :

            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_limit = Q1 - 1.5*IQR
            upper_limit = Q3 + 1.5*IQR

            outlier_index = df[(df[col] < lower_limit) | (df[col] > upper_limit)].index
            outliers_index_list.extend(outlier_index.tolist())
        else:
            continue

    unique_outliers_index = list(set(outliers_index_list))
    df = df.drop(unique_outliers_index, axis="rows")
    print("Total Outliers Removed: ",  len(unique_outliers_index))
    return df

In [ ]:
# Remove Outliers Using Standard Deviation for multiple columns

def Remove_outliers_using_STD(df):
    outliers_index_list = []

    for col in df.columns:
        if df[col].dtypes == "float64":

            lower_limit = df[col].mean() - 3*df[col].std()
            upper_limit = df[col].mean() + 3*df[col].std()

            outlier_index = df[(df[col] < lower_limit) | (df[col] > upper_limit)].index
            outliers_index_list.extend(outlier_index.tolist())
        else:
            continue

    unique_outliers_index = list(set(outliers_index_list))
    df = df.drop(unique_outliers_index, axis="rows")
    print("Total Outliers Removed: ",  len(unique_outliers_index))
    return df

In [ ]:
# Remove Specific Outliers using percentile

min_threshold, max_threshold = df['column_name'].quantile([0.007, 0.99])
new_df = df[(df['column_name'] > min_threshold) & (df['column_name'] < max_threshold)]

display(df.sort_values(by="column_name", ascending=True))
print("df Rows:", df.shape[0])
print("Total Rows Removed:", df.shape[0] - new_df.shape[0])

## Capping or Winsorization

In [ ]:
# Replace the outliers with the non outliers value

lower_limit = df['column_name'].quantile(0.05)
upper_limit = df['column_name'].quantile(0.95)

df['column_name'] = np.where( df['column_name'] >= upper_limit,
                              upper_limit,
                              np.where(df['column_name'] <= lower_limit,
                              lower_limit,
                              df['column_name']))